In [21]:
#SETTINGS and imports

from settings.settings import data_settings

import pandas as pd
import numpy as np
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 200
pd.options.display.max_columns = 1000


In [43]:
name = data_settings.data_name.replace("_R","")
df = pd.read_csv(f"{data_settings.csv_path}/with_class/{name}_slatr_{data_settings.sl_to_atr:02}.csv")
# df.fillna(0,inplace=True)
df["DateTime"] = pd.to_datetime(df.pop("DateTime"))

usefulls = ["Open", "High", "Low", "Close", "TickVol",
            "Spread", "buy_1", "buy_1.25", "buy_1.5",
            "buy_1.75", "buy_2", "buy_4", "buy_6","sell_1",
            "sell_1.25", "sell_1.5", "sell_1.75", "sell_2",
            "sell_4", "sell_6", "do_nothing", "windowsizes", "DateTime"]

for column in df.columns:
    if column not in usefulls:
        df.pop(column)
        
df.High = df.High.sub(df.Close.shift(1))
df.Open = df.Open.sub(df.Close.shift(1))
df.Low = df.Low.sub(df.Close.shift(1))
df.Close = df.Close.diff()

df.Close.replace(0,1e-15,inplace=True)

df.to_csv(f"{data_settings.csv_path}/cleaned_mt_data/{data_settings.data_name}.csv",index = False)


In [36]:
df.loc[df.Close == 0]

,Open,High,Low,Close,TickVol,Spread,buy_1,buy_1.25,buy_1.5,buy_1.75,buy_2,buy_4,buy_6,sell_1,sell_1.25,sell_1.5,sell_1.75,sell_2,sell_4,sell_6,do_nothing,windowsizes,DateTime


In [42]:
df = pd.read_csv(f"{data_settings.csv_path}/cleaned_mt_data/{data_settings.data_name}.csv")
# df.fillna(0,inplace=True)
df["DateTime"] = pd.to_datetime(df.pop("DateTime"))
df.loc[df.Close == 0]

,Open,High,Low,Close,TickVol,Spread,buy_1,buy_1.25,buy_1.5,buy_1.75,buy_2,buy_4,buy_6,sell_1,sell_1.25,sell_1.5,sell_1.75,sell_2,sell_4,sell_6,do_nothing,windowsizes,DateTime


In [ ]:
# df.Close.plot()
# df.High.plot()
# df.Low.plot()
# df.Open.plot()

In [ ]:
import seaborn as sns
import matplotlib as mpl
import ta
mpl.rcParams["figure.figsize"]= (40,20)
sns.set_theme(style="darkgrid")

i = 5
df["A"] = df.Close.diff()
df["A_EMA"] = ta.trend.ema_indicator(df["A"],24)
a = df[-2000:]
data_preproc = a[["DateTime","A_EMA"]]
 
g = sns.relplot(x="DateTime", y="value", data=pd.melt(data_preproc, ['DateTime']),hue="variable",height=10,aspect=2,kind="line")#,style="buy_1"

g.figure.autofmt_xdate()

In [ ]:
df.AVERAGE_TRUE_RANGE.describe().transpose()


In [ ]:
import seaborn as sns
import matplotlib as mpl
mpl.rcParams["figure.figsize"]= (40,20)
sns.set_theme(style="darkgrid")


key = "buy_1.25"
# g = sns.relplot(x="DateTime", y="Close", data=df,height=10,aspect=2,style="buy_1")#kind="line",
g=sns.scatterplot(data=df, x="DateTime", y="Close", hue=key)
g.figure.autofmt_xdate()

# key = "buy_2"
# g=sns.scatterplot(data=df, x="DateTime", y="Close", hue=key)
# g.figure.autofmt_xdate()


In [ ]:
i = 1.75
print("All",df.shape[0])
print("Do nothings:",df.loc[df[f"sell_{i}"] == 0].loc[df[f"buy_{i}"] ==0].shape[0])
print("Sells:",df.loc[df[f"sell_{i}"] == 1].shape[0])
print("Buys:",df.loc[df[f"buy_{i}"] == 1].shape[0])


In [ ]:
print(df.loc[df["sell_2"]==1].loc[df["sell_1"]==1].shape[0])

In [ ]:
# LOADING DATA
# df_with_target = pd.read_csv(data_settings.data_with_target_path)
# df_with_indicators = pd.read_csv(data_settings.data_with_indicators_path)
df_final_not_cleaned = pd.read_csv(data_settings.data_final_not_cleaned_path)
df_final_not_cleaned

In [ ]:
# MAKING A READY DF
df = df_final_not_cleaned.copy()

# df.pop(df.PSAR_DOWN.name)
# df.pop(df.PSAR_UP.name)
# df.pop(df.KAMA.name)
df.pop(df.AVERAGE_TRUE_RANGE.name)
df=df.dropna().reset_index(drop=True)

date_time = pd.to_datetime(df.pop("DateTime"),format=data_settings.date_time_format)
timestamps = date_time.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day
df["Day_sin"] = (np.sin(timestamps * (2 * np.pi / day))+1)/2
df["Day_cos"] = (np.cos(timestamps * (2 * np.pi / day))+1)/2
df["Year_sin"] = (np.sin(timestamps * (2 * np.pi / year))+1)/2
df["Year_cos"] = (np.cos(timestamps * (2 * np.pi / year))+1)/2

# plt.plot(np.array(df["Day_sin"])[:1000])
# plt.plot(np.array(df["Day_cos"])[:1000])
# plt.plot(np.array(df["Year_sin"])[:1000])
# plt.plot(np.array(df["Year_cos"])[:1000])

#make targets
df.pop(df.buy_1.name)
df.pop(df.buy_2.name)
df.pop(df.buy_8.name)
df.pop(df.buy_16.name)

df.pop(df.sell_1.name)
df.pop(df.sell_2.name)
df.pop(df.sell_8.name)
df.pop(df.sell_16.name)
df.pop(df.windowsizes.name)
df.pop(df.do_nothing.name)

do_nothings = []
for i in range(len(df)):
    if(df.buy_4.iloc[i]==0 and df.sell_4.iloc[i]==0):
        do_nothings.append(1)
    else:
        do_nothings.append(0)
        
df["do_nothing"]= do_nothings
df["DateTime"]= date_time

df.to_csv(data_settings.data_final_path,index=False)

In [ ]:
# Split to train and test

n=len(df)
train_df = df[:int((0.85)*n)].reset_index(drop=True)
test_df = df[int((0.85)*n):].reset_index(drop=True)
train_df.to_csv(data_settings.data_train_path,index=False)
test_df.to_csv(data_settings.data_test_path,index=False)

In [ ]:
# train = pd.read_csv(data_settings.data_train_path)
# test = pd.read_csv(data_settings.data_test_path)

In [ ]:
train_df = pd.read_csv(data_settings.data_train_path)
test_df = pd.read_csv(data_settings.data_test_path)
window_size = data_settings.window_size

test_buy = test_df.pop(test_df.buy_4.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.sell_4.name)

train_buy = train_df.pop(train_df.buy_4.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.sell_4.name)

In [ ]:
test_labels = (
                test_buy.to_numpy(),
                test_nothing.to_numpy(),
                test_sell.to_numpy()
               )
test_labels = np.array(test_labels)
test_labels = test_labels.transpose()
test_labels = test_labels.astype("float32")

train_labels = (
                train_buy.to_numpy(),
                train_nothing.to_numpy(),
                train_sell.to_numpy()
               )
train_labels = np.array(train_labels)
train_labels = train_labels.transpose()
train_labels = train_labels.astype("float32")

In [ ]:
def x_train_generator():
    for i in range(window_size,len(train_df)):
        window=train_df[i-window_size:i]
        yield ((window-window.min())/(window.max()-window.min())).to_numpy()

def x_test_generator():
    for i in range(window_size,len(test_df)):
        window=test_df[i-window_size:i]
        yield ((window-window.min())/(window.max()-window.min())).to_numpy()    
        
def y_train_generator():
    for i in range(window_size-1,train_labels.shape[0]):
        yield train_labels[i]

def y_test_generator():
    for i in range(window_size-1,test_labels.shape[0]):
        yield test_labels[i]
        
x_train_ds = tf.data.Dataset.from_generator(x_train_generator
                                        output_signature=(
                                            tf.TensorSpec(shape=(256,101), dtype=tf.float64)
                                            )
                                        ).batch(256)

x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
                                        output_signature=(
                                            tf.TensorSpec(shape=(256,101), dtype=tf.float64)
                                            )
                                        ).batch(256)
y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
                                        output_signature=(
                                            tf.TensorSpec(shape=(3), dtype=tf.float32)
                                            )
                                        ).batch(256)
y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
                                        output_signature=(
                                            tf.TensorSpec(shape=(3), dtype=tf.float32)
                                            )
                                        ).batch(256)

train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

In [ ]:
def money(initial,percent,month):
    
    if month ==0 :
        return initial
    
    else:
        return percent * money(initial,percent,month-1) + money(initial,percent,month-1)
    
